# 1. Carga Dataset

En esta sección, se implementa la descarga del conjunto de datos siguiendo las especificaciones establecidas en el enunciado del taller. Para ello, se utilizan las bibliotecas **os** y **requests**, encargadas de gestionar el sistema de archivos y realizar solicitudes HTTP, respectivamente. Gracias a esta combinación de herramientas, se automatiza la recuperación del conjunto de datos desde una fuente remota, garantizando su disponibilidad para el desarrollo y ejecución del experimento.

In [1]:
import os
import requests

# Directorio para descargar el conjunto de datos
_data_root = './data/covertype'
# Path del archivo para guardarlo dentro del ambiente
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

# Descarga del conjunto de datos
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    url = 'https://docs.google.com/uc?export=\download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


Para la lectura y manipulación del conjunto de datos dentro del entorno de JupyterLab, se utiliza la biblioteca **pandas**, ampliamente reconocida en la comunidad científica por sus capacidades de procesamiento y análisis de datos estructurados. Adicionalmente, se incorpora el atributo **display** de la biblioteca **IPython**, el cual optimiza la representación tabular de los datos, facilitando su inspección y exploración de manera más clara e intuitiva. Esta estrategia permite una interacción eficiente con el conjunto de datos a lo largo del desarrollo del taller. 

In [2]:
import pandas as pd 
from IPython.display import display 

# Lectura y visualización rápida del conjunto de datos
df_inicial=pd.read_csv("data/covertype/covertype_train.csv")
print(display(df_inicial))

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116198,3150,220,16,285,47,2275,200,253,187,866,Commanche,C7756,1
116199,3125,47,13,234,2,2430,224,212,120,1426,Rawah,C7745,0
116200,3166,152,11,67,0,1275,234,240,136,2404,Rawah,C7202,0
116201,3154,285,14,738,46,6012,181,239,198,1320,Rawah,C7745,1


None


# 2. Selección de Características

Se realiza una caracterización detallada del conjunto de datos mediante la enumeración de sus variables, junto con el tipo de datos asociado a cada una de ellas. Además, se computa el número de valores nulos presentes en cada columna con el propósito de identificar posibles inconsistencias o datos faltantes. Para ello, se emplea el método **.info()** de la estructura **DataFrame** de la biblioteca **pandas**, el cual proporciona un resumen técnico del esquema de datos, facilitando el análisis exploratorio inicial y la detección de posibles necesidades de preprocesamiento.

In [3]:
print(df_inicial.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

Se procede a segmentar el conjunto de datos en tres subconjuntos diferenciados, con el objetivo de estructurar de manera eficiente las variables en función de su tipo y su rol en el análisis:

-  **Subconjunto de variables numéricas:** Se extraen exclusivamente las columnas que contienen datos de tipo numérico, facilitando su posterior tratamiento en técnicas de normalización, escalamiento y modelado estadístico o basado en aprendizaje automático.

-  **Subconjunto de variables categóricas:** Se agrupan las columnas que representan información cualitativa, lo que permitirá la aplicación de estrategias de codificación adecuadas en el preprocesamiento.

-  **Vector de la variable objetivo:** Se aísla la columna que contiene la variable de interés, la cual se pretende predecir en el modelo. Esta segmentación es fundamental para estructurar correctamente la entrada y salida del sistema de modelado.

Este proceso de partición facilita el diseño de estrategias de preprocesamiento y modelado diferenciadas según la naturaleza de los datos, optimizando la eficiencia y la interpretabilidad del análisis.

In [4]:
# Nombre de la variable objetivo
target_col = 'Cover_Type'

# Subconjunto de columnas numericas
df_inicial_numeric = df_inicial.select_dtypes(include=["number"]).drop(columns=[target_col], errors="ignore")

# Subconjunto de columnas no numericas
df_inicial_categoric = df_inicial.select_dtypes(include=["object"]).drop(columns=[target_col], errors="ignore")

# Vector con la variable de interes
target = df_inicial[ target_col ].astype("category")

# Visualización de las categorías de la variable objetivo
target.value_counts().to_frame(name="Frecuencia").style.set_table_attributes("style='display:inline'").set_caption("Distribución de Cover Type")

,Frecuencia
1,56720
0,42307
2,7228
6,4045
5,3478
4,1892
3,533


Se incorpora la biblioteca **NumPy**, una de las herramientas fundamentales en el ecosistema de ciencia de datos, debido a su eficiencia en la manipulación de estructuras matriciales y la optimización de cálculos numéricos de alto rendimiento.

Adicionalmente, se importan los módulos **SelectKBest** y **f_classif** de la biblioteca **Scikit-Learn**, los cuales permiten realizar una selección de características univariada. Este proceso se basa en la evaluación individual de cada variable predictora en función de su relación estadística con la variable objetivo, utilizando pruebas de ANOVA (Análisis de Varianza) en el caso de problemas de clasificación.

La combinación de estas herramientas resulta esencial para la reducción de dimensionalidad, mejorando la eficiencia computacional del modelo y minimizando el ruido en los datos por posibles conflictos de multicolinealidad.


In [5]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif

# Número de categorias en la variable objetivo
k = 7  

# Aplicar SelectKBest con f_classif (ANOVA F-score)
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(df_inicial_numeric, target)

# Obtener los puntajes de cada característica
scores = selector.scores_

# Crear un DataFrame con los resultados
feature_scores = pd.DataFrame({
    'Feature': df_inicial_numeric.columns,
    'Score': scores
}).sort_values(by='Score', ascending=False)

# Imprimir los resultados
print("Puntajes de características numéricas:")
print(display(feature_scores))

Puntajes de características numéricas:


,Feature,Score
0,Elevation,31087.079657
5,Horizontal_Distance_To_Roadways,2050.314700
2,Slope,1559.369722
9,Horizontal_Distance_To_Fire_Points,1452.737911
6,Hillshade_9am,643.862634
7,Hillshade_Noon,595.275348
3,Horizontal_Distance_To_Hydrology,488.221292
4,Vertical_Distance_To_Hydrology,241.029561
8,Hillshade_3pm,190.497164
1,Aspect,88.987497


None


Se genera una representación tabular en la que se visualizan las variables del conjunto de datos junto con un indicador booleano que señala cuáles han sido seleccionadas para su inclusión en las siguientes fases del experimento. Esta tabla permite identificar de manera clara y estructurada las variables numéricas que han sido consideradas relevantes en el proceso de selección de características.

La visualización de esta información facilita la validación del procedimiento de filtrado y optimización de las variables predictoras, asegurando que únicamente aquellas con mayor aporte estadístico sean utilizadas en el modelado subsecuente.

In [6]:
# listar las k=7 variables seleccionadas del proceso ya descrito
selected_features = df_inicial_numeric.columns[selector.get_support()]

# Representación booleana de la selección de variables numéricas
columns_selected = pd.DataFrame({
    'Columns': df_inicial_numeric.columns,
    'Retain': [i in selected_features.tolist() for i in df_inicial_numeric.columns]
})#.sort_values(by='Score', ascending=False)
print(display(columns_selected))

,Columns,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,False
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


None


Se lleva a cabo una previsualización del subconjunto de datos conformado por las variables numéricas seleccionadas mediante el proceso de filtrado, en conjunto con las variables categóricas y la variable objetivo. Esta inspección preliminar permite verificar la correcta estructuración del conjunto de datos y evaluar posibles relaciones entre las distintas variables.

La integración de variables de diferentes tipos en esta fase facilita la identificación de patrones subyacentes y proporciona una base sólida para los posteriores procedimientos de preprocesamiento y modelado.

Adicional a esto, se realiza una corrección en la interpretación de la variable objetivo, ya que el framework **TFX** la identifica erróneamente como una variable numérica. Sin embargo, dado el contexto del problema, se reconoce que dicha variable es de naturaleza categórica. Este ajuste es crucial, ya que una clasificación incorrecta de la variable de interés puede afectar negativamente la configuración del pipeline de procesamiento, comprometiendo la correcta aplicación de técnicas de transformación y modelado. Por lo tanto, tras detectar esta discrepancia durante la experimentación, se decide retroceder a esta fase y realizar la conversión adecuada, garantizando así que la variable sea tratada de acuerdo con su verdadera naturaleza.

In [7]:
# Selección de las variables numéricas filtradas
df_inicial_numeric = df_inicial_numeric[selected_features.tolist()]

# Concatenación de las variables en un sólo dataframe
df_variables = pd.concat([df_inicial_numeric, df_inicial_categoric, target], axis=1)

# Hacemos una corrección dentro de la interpretación de la variable de interés dentro de la máquina
df_variables['Cover_Type'] = df_variables['Cover_Type'].astype(str).apply(lambda x: f"_{x}")
print(display(df_variables))

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,1015,233,234,1570,Commanche,C7202,_1
1,2876,18,485,2495,192,202,1557,Commanche,C7757,_1
2,3171,2,277,4374,213,237,1052,Rawah,C7745,_0
3,3087,13,190,4774,193,221,752,Rawah,C7745,_0
4,2835,10,212,3596,231,242,3280,Rawah,C4744,_1
...,...,...,...,...,...,...,...,...,...,...
116198,3150,16,285,2275,200,253,866,Commanche,C7756,_1
116199,3125,13,234,2430,224,212,1426,Rawah,C7745,_0
116200,3166,11,67,1275,234,240,2404,Rawah,C7202,_0
116201,3154,14,738,6012,181,239,1320,Rawah,C7745,_1


None


# 3. Data Pipeline

## 3.1 Configurar el Contexto Interactivo

Se procede a la importación de los módulos esenciales para la construcción y ejecución del pipeline utilizando TFX (TensorFlow Extended). En esta fase, se incorporan las siguientes dependencias:

-  **tensorflow:** Biblioteca base para la construcción y ejecución de modelos de aprendizaje automático.

-  **tfx.components:** Conjunto de componentes modulares de TFX que facilitan la ingesta, validación, transformación y análisis de datos.  

-  **CsvExampleGen:** Permite la conversión de datos en formato CSV a TFRecords, formato optimizado para TensorFlow.

-  **ExampleValidator:** Detecta anomalías y valores atípicos en los datos.

-  **SchemaGen:** Genera un esquema inferido a partir de los datos, facilitando la detección de inconsistencias.

-  **StatisticsGen:** Calcula estadísticas descriptivas sobre el conjunto de datos.

-  **Transform:** Facilita la ingeniería de características mediante el uso de TensorFlow Transform.

-  **interactive_context:** Permite la ejecución del pipeline en un entorno interactivo, como JupyterLab, lo que facilita su depuración y análisis en tiempo real.

**Nota sobre advertencias (warnings) del sistema**

A partir de este punto, comenzarán a generarse advertencias relacionadas con la ausencia de controladores de GPU en el entorno de ejecución. Esto ocurre porque **TFX** y **TensorFlow** están optimizados para aprovechar la aceleración por hardware mediante GPU, lo que mejora significativamente los tiempos de procesamiento. Sin embargo, dado que el presente experimento se ejecutará en CPU, estas advertencias no afectan la funcionalidad ni la reproducibilidad del análisis.


In [8]:
import tensorflow as tf
# Importar componentes de TFX
from tfx.components import CsvExampleGen, ExampleValidator, SchemaGen, StatisticsGen, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint
pp = pprint.PrettyPrinter()

2025-03-04 02:27:30.180460: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-04 02:27:30.185487: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-04 02:27:30.195187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-04 02:27:30.209958: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-04 02:27:30.210060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 02:27:30.223438: I tensorflow/core/platform/cpu_feature_guard.cc:

Se procede a la definición del **contexto interactivo** de **TFX**, el cual permite la ejecución progresiva y modular del pipeline dentro del entorno de experimentación. En esta configuración, se establece un directorio específico de almacenamiento, representado por la variable **_pipeline_root**, el cual servirá como repositorio persistente para todos los artefactos generados durante el proceso, incluyendo los ejemplos procesados, esquemas inferidos, estadísticas de datos y transformaciones aplicadas.

En caso de omitir la definición del **pipeline_root**, el contexto interactivo se ejecutaría de manera flotante, lo que implicaría la pérdida de la trazabilidad del experimento, ya que los artefactos generados no serían almacenados de forma persistente dentro del volumen asignado. Al establecer explícitamente esta ruta de almacenamiento, se garantiza la reproducibilidad del experimento y la correcta gestión de los metadatos asociados a cada etapa del pipeline.

In [9]:
_pipeline_root = './pipeline/'
context = InteractiveContext(pipeline_root=_pipeline_root)

## 3.2. Generando Ejemplos

Antes de proceder con la ingesta de datos en **TFX**, es necesario estructurar y almacenar la información en un archivo **CSV** compatible con el framework. Este paso garantiza que los datos estén organizados en un formato tabular adecuado, permitiendo su procesamiento eficiente dentro del pipeline de **TFX**.

El uso de **CSV** como formato de almacenamiento facilita la interoperabilidad con el componente **CsvExampleGen**, el cual convierte estos datos en **TFRecords**, optimizando su manipulación dentro del ecosistema de **TensorFlow**. Además, este procedimiento permite asegurar que el conjunto de datos contenga la estructura y formato adecuados para ser correctamente interpretados por los componentes posteriores del **pipeline**.

In [10]:
# Definir la ruta donde se guardará el archivo CSV
data_dir = os.path.join(os.getcwd(), "data_tfx")
os.makedirs(data_dir, exist_ok=True)
csv_path = os.path.join(data_dir, "data.csv")

# Guardar el DataFrame como CSV (sin el índice para evitar problemas)
df_variables.to_csv(csv_path, index=False)

Se lleva a cabo la conversión del conjunto de datos desde su formato **CSV** a **TFRecords** mediante el uso del componente **CsvExampleGen** de **TFX**. Este proceso es fundamental, ya que **TFRecords** es un formato de almacenamiento optimizado para TensorFlow, permitiendo una lectura más eficiente y escalable en comparación con archivos **CSV** tradicionales.

Para materializar la ingesta de datos en el **pipeline**, es imprescindible ejecutar el método **.run()**, el cual instancia el objeto **example_gen** y procede con la transformación de los datos, almacenando los ejemplos convertidos en el directorio de salida correspondiente. De este modo, se garantiza que la información se encuentre disponible para las etapas subsecuentes del **pipeline**, como la generación de estadísticas, validación de datos y preprocesamiento.

In [11]:
example_gen = CsvExampleGen(input_base=data_dir)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se procede a la recuperación del artefacto generado por el componente **CsvExampleGen**, permitiendo la verificación de los nombres de los subconjuntos en los que se ha particionado el conjunto de datos. Este proceso es clave para corroborar que la segmentación de los datos (por ejemplo, en entrenamiento y validación) se haya realizado conforme a los parámetros predefinidos en la canalización.

Adicionalmente, se inspecciona la ruta de almacenamiento donde se han alojado los datos procesados dentro del **pipeline**, lo que permite rastrear la ubicación exacta de los artefactos generados. Este paso es esencial para la trazabilidad del flujo de datos y facilita su reutilización en las etapas posteriores del experimento.

In [12]:
artifact = example_gen.outputs['examples'].get()[0]
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


Se procede a la extracción de la ruta específica correspondiente al subconjunto de entrenamiento dentro del **artefacto** previamente almacenado. Este paso es fundamental, ya que permite acceder de manera directa y estructurada a los datos procesados, evitando la necesidad de manipular el artefacto completo en cada etapa del **pipeline**.

In [13]:
train_uri = os.path.join(artifact.uri, 'Split-train')
!ls {train_uri} # usamos ! para que el comando no corra sobre python, si no sobre la imagen de linux sobre el cual está montado docker

data_tfrecord-00000-of-00001.gz


Se procede a la carga de los ejemplos correspondientes al subconjunto de entrenamiento utilizando la clase **TFRecordDataset** de TensorFlow. Este enfoque permite acceder de manera eficiente a los datos almacenados en formato **TFRecords**, optimizado para el procesamiento de grandes volúmenes de datos en entornos de aprendizaje profundo.

La utilización de **TFRecordDataset** facilita la lectura secuencial y el acceso estructurado a la información contenida en los registros, permitiendo su posterior transformación y análisis dentro del pipeline de TFX. Este paso es importante para garantizar que los datos sean correctamente interpretados en las etapas de validación, preprocesamiento y modelado.

In [14]:
# Obtenemos la lista de objetos guardados dentro del subconjunto de entrenamiento
tfrecord_filenames = [
    os.path.join(train_uri, name) for name in os.listdir(train_uri)
]
# Creamos `TFRecordDataset` para leer directamente la lista de archivos almacenados
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

Se implementa una función que permite visualizar los n primeros registros contenidos en el **TFRecordDataset**, facilitando la inspección y validación de la estructura de los datos almacenados en este formato. Esta función es particularmente útil para garantizar que los ejemplos hayan sido correctamente serializados y puedan ser interpretados de manera adecuada en las siguientes etapas del **pipeline**.

A continuación, se procede a la recuperación y visualización de tres ejemplos, permitiendo examinar el contenido de los datos y asegurando su correcta correspondencia con la información original del conjunto de entrenamiento. Este proceso es esencial para verificar la integridad de los datos procesados antes de avanzar en el flujo de la canalización.

In [15]:
# Definición de función auxiliar para la recuperación de una muestra de n registros del TFRecords 
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    # Definición de una lista vacía
    records = []
    # uso del método .take para extraer la muestra
    for tfrecord in dataset.take(num_records):
        # Se obtiene el tensor del ejemplo recuperado
        serialized_example = tfrecord.numpy()
        # Se crea el lector de objetos serializados
        example = tf.train.Example()
        # Se obtiene la información del tensor (output es un protocol buffer message)
        example.ParseFromString(serialized_example)
        # Se convierte el protocol bufffer message a un diccionario de Python
        example_dict = (MessageToDict(example))
        # Adjuntamos el ejemplo a la lista de ejemplos recuperados
        records.append(example_dict)
    return records

sample_records = get_records(dataset, 3)
pp.pprint(sample_records)    

[{'features': {'feature': {'Cover_Type': {'bytesList': {'value': ['XzE=']}},
                           'Elevation': {'int64List': {'value': ['2991']}},
                           'Hillshade_9am': {'int64List': {'value': ['233']}},
                           'Hillshade_Noon': {'int64List': {'value': ['234']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1570']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['67']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['1015']}},
                           'Slope': {'int64List': {'value': ['7']}},
                           'Soil_Type': {'bytesList': {'value': ['QzcyMDI=']}},
                           'Wilderness_Area': {'bytesList': {'value': ['Q29tbWFuY2hl']}}}}},
 {'features': {'feature': {'Cover_Type': {'bytesList': {'value': ['XzE=']}},
                           'Elevation': {'int64List': {'value': ['2876']}

2025-03-04 02:27:56.245055: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 3.3. Estadísticas

A partir del artefacto generado por **CsvExampleGen**, se procede a la creación e implementación del componente **StatisticsGen** dentro del **pipeline** de TFX. Este componente es responsable de calcular estadísticas descriptivas sobre el conjunto de datos, permitiendo analizar la distribución de las variables, detectar valores atípicos y evaluar la presencia de datos faltantes.

Una vez instanciado el artefacto, se ejecuta dentro del pipeline para generar y almacenar las estadísticas en el sistema de metadatos. Estas métricas son fundamentales para la validación y preprocesamiento de los datos, proporcionando información clave para la detección de posibles anomalías y la toma de decisiones en el flujo de transformación y modelado del experimento.

In [16]:
# Generar el artefacto StatisticGen a partir del CsvExampleGen generado
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Ejecutar el artefacto
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se procede a la visualización de los resultados generados por el artefacto **StatisticsGen**, lo que permite realizar una inspección detallada de las estadísticas calculadas sobre el conjunto de datos. Esta revisión es fundamental para evaluar la calidad de los datos, identificar posibles inconsistencias y detectar **anomalías** en las distribuciones de las variables.

El análisis de estas métricas facilita la toma de decisiones en etapas posteriores del **pipeline**, tales como la validación de datos con **ExampleValidator** y la ingeniería de características con **Transform**. De esta manera, se asegura que el conjunto de datos procesado sea coherente y adecuado para el desarrollo de modelos de aprendizaje automático dentro del entorno de **TFX**.

In [17]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

## 3.4. Inferir el esquema

Para poder inferir el esquema, primero debemos inicializarlo con el artefacto SchemaGen

Para proceder con la inferencia del esquema del conjunto de datos, se debe inicializar el componente **SchemaGen** dentro del **pipeline** de TFX. Este artefacto se encarga de analizar las estadísticas generadas previamente por **StatisticsGen** y, con base en ellas, inferir automáticamente un esquema que describe la estructura de los datos, incluyendo los tipos de variables, dominios de valores y restricciones de integridad.

La correcta definición del esquema es fundamental para la detección de anomalías y validaciones en las etapas posteriores del **pipeline**. Además, proporciona una referencia estructurada que facilita el diseño de transformaciones y garantiza la coherencia de los datos en el flujo de procesamiento.

In [18]:
# Generamos el artefacto SchemaGen, este se puede interpretar como la inicialización del esquema
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )
# Ejecutamos la inicialización del artefacto
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Tras la ejecución del componente **SchemaGen**, se procede a la revisión del esquema inferido para evaluar su correspondencia con la estructura esperada del conjunto de datos. En este análisis, se verifica que la clasificación automática de las variables sea coherente con la naturaleza de los datos, asegurando que las variables categóricas y numéricas hayan sido correctamente identificadas.

En este caso particular, se observa que el esquema inferido cumple con las expectativas del experimento: tres variables han sido categorizadas como cualitativas, mientras que el resto han sido reconocidas como numéricas. Esta validación es crucial, ya que un esquema correctamente definido garantiza la adecuada aplicación de transformaciones y validaciones en las siguientes etapas del **pipeline**.

Además, diferenciamos que el carácter especial incluido dentro de las entradas de la variable **Cover_Type**, forzando que esta sea reconocida como cualitativa, es decir, se reconozca como una variable no numérica. 

In [19]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 3.5. Curando el Esquema

Se procede a la carga del esquema inferido utilizando la biblioteca **TFDV** (**TensorFlow Data Validation**), con el objetivo de realizar una revisión detallada, y si es necesario, ajustar su configuración para que se alinee con el conocimiento previo sobre las características del conjunto de datos.

Este proceso permite corregir posibles inconsistencias en la inferencia automática, tales como la clasificación incorrecta de tipos de variables, la definición de rangos inapropiados o la ausencia de restricciones esperadas en los datos. La capacidad de modificar el esquema asegura que el **pipeline** de TFX interprete correctamente la estructura de los datos, facilitando la validación y transformación de las características en etapas posteriores.

In [20]:
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

# Obtener la ruta del esquema
schema_uri = schema_gen.outputs['schema'].get()[0].uri

# Cargar el esquema en memoria
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")

Se procede a la adecuación de los rangos de valores para tres variables específicas del conjunto de datos, garantizando que su representación en el esquema sea consistente con los valores reales que pueden asumir. Esta modificación es crucial para evitar inconsistencias en la validación de datos y asegurar la correcta interpretación de las características dentro del pipeline de TFX.

Las correcciones aplicadas son las siguientes:

-  **Hillshade 9am:** Se establece un rango válido entre 0 y 255, dado que esta variable representa valores de sombreado en función de la posición del sol en la mañana.

-  **Hillshade Noon:** Se ajusta al rango 0 a 255, correspondiente a las condiciones de iluminación al mediodía.

-  **Slope:** Se define un límite entre 0 y 90 grados, reflejando los valores posibles de inclinación del terreno en grados.

Estas modificaciones dentro del SchemaGen garantizan que el flujo de datos dentro del pipeline respete las restricciones establecidas, minimizando la posibilidad de generar alertas por valores fuera de rango y mejorando la calidad del preprocesamiento y validación de datos. 

**Nota:** Sobre la variable Cover_Type, no se hace adecuaciones, pues hemos modificado desde el principio de la construcción del **pipeline** como variable categórica, y dentro del esquema inferido se encuentra de manera correcta el rango para esta variable.

In [21]:
# Arreglo del rango de Hillshade_9am
Hillshade_9am_domain = schema_pb2.IntDomain(
    name='Hillshade_9am_domain',min=0,max=255 )
tfdv.set_domain(schema, 'Hillshade_9am', Hillshade_9am_domain)

# Arreglo del rango de Hillshade_Noon
Hillshade_Noon_domain = schema_pb2.IntDomain(
    name='Hillshade_Noon_domain',min=0,max=255 )
tfdv.set_domain(schema, 'Hillshade_Noon', Hillshade_Noon_domain)

# Arreglo del rango de Slope
slope_domain = schema_pb2.IntDomain(
    name='slope_domain', min=0, max=90
)
tfdv.set_domain(schema, 'Slope', slope_domain)


Se procede a la visualización del artefacto **SchemaGen** curado, el cual ha sido ajustado conforme a los conocimientos previos sobre la estructura y restricciones de las variables del conjunto de datos. Esta inspección permite verificar que las modificaciones realizadas en los rangos de valores y la categorización de las variables hayan sido correctamente aplicadas dentro del esquema inferido.

La validación del esquema curado es un paso esencial dentro del **pipeline**, ya que garantiza que los datos sean interpretados de manera precisa en las etapas posteriores, como la detección de anomalías, la transformación de características y la optimización del modelo. Además, asegura que los artefactos generados sean compatibles con la infraestructura de TFX, promoviendo la reproducibilidad y consistencia del flujo de trabajo en producción.

In [22]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


Se procede al almacenamiento del artefacto **SchemaGen curado**, asegurando que las modificaciones realizadas sobre la estructura de los datos sean persistentes y reproducibles dentro del **pipeline** de TFX. Este proceso garantiza que los ajustes aplicados en los tipos de datos, rangos de valores y restricciones sean conservados para su uso en futuras ejecuciones del pipeline, evitando la necesidad de redefinir manualmente estos parámetros en cada iteración del experimento.

El guardado del esquema actualizado permite su reutilización en etapas posteriores, como la validación de datos con **ExampleValidator** y la transformación de características con **Transform**. Además, este artefacto servirá como referencia para asegurar la coherencia en la estructura de los datos de entrada, tanto en entrenamiento como en producción, minimizando el riesgo de errores en el flujo de datos.

In [23]:
tfdv.write_schema_text(schema, schema_uri + "/schema.pbtxt")

## 3.6. Entornos de Esquema

Se inicia el proceso de definición de entornos del esquema con el objetivo de diferenciar el uso de los datos en las distintas fases del **pipeline**. Para ello, se realiza la carga desde cero del esquema curado, asegurando que las modificaciones previamente aplicadas sean efectivas en la configuración del flujo de datos.

Posteriormente, se crean los entornos específicos:

* **TRAINING**: Entorno que conserva todas las variables del conjunto de datos, incluyendo la variable objetivo Cover_Type, utilizada para el entrenamiento del modelo.

* **SERVING**: Entorno diseñado para la validación y despliegue del modelo, donde se eliminan las variables que no estarán disponibles en producción.

En este caso particular, se elimina la variable **Cover_Type** del entorno SERVING, dado que esta representa la variable objetivo que el modelo debe predecir. Esta eliminación es fundamental para simular un entorno de producción realista, donde el modelo deberá inferir dicha variable en función de las características proporcionadas en los datos de entrada.

La correcta definición de estos entornos garantiza que el esquema de datos refleje las condiciones tanto de entrenamiento como de inferencia, permitiendo una evaluación rigurosa del modelo y asegurando la coherencia en el flujo de datos a lo largo del pipeline.

In [24]:
# Cargar el esquema generado previamente
schema_uri = schema_gen.outputs['schema'].get()[0].uri
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")

# Configurar los entornos del esquema
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# Remover Cover_type del entorno SERVING
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

Guardamos el esquema con los entornos configurados

Se procede al almacenamiento del esquema con los entornos configurados, asegurando que las modificaciones aplicadas en la estructura de los datos sean persistentes y reproducibles en futuras ejecuciones del **pipeline**.

La correcta configuración y almacenamiento de los entornos (**TRAINING** y **SERVING**) permite que el flujo de datos refleje de manera consistente las condiciones esperadas en cada fase del pipeline, diferenciando el conjunto de variables utilizadas durante el entrenamiento y las que estarán disponibles en producción.

Este paso es fundamental para garantizar que el esquema curado sea accesible en etapas posteriores del proceso, tales como la validación de datos, la transformación de características y la evaluación del modelo, contribuyendo a la trazabilidad y estabilidad del pipeline en un entorno de producción realista.

In [25]:
tfdv.write_schema_text(schema, schema_uri + "/schema.pbtxt")

Se procede a la visualización del esquema actualizado, que ahora incluye la configuración de los entornos TRAINING y SERVING. Esta inspección es crucial para verificar que la separación de los entornos se haya realizado correctamente, asegurando que la variable objetivo (Cover_Type) permanezca en el entorno de entrenamiento y sea excluida del entorno de servicio.

In [26]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Cover_Type',"'_0', '_1', '_2', '_3', '_4', '_5', '_6'"
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


Se lleva a cabo la verificación final de la configuración de los entornos TRAINING y SERVING, asegurando que la estructura de datos definida en el esquema se haya aplicado correctamente.

Como parte de este proceso de validación, se confirma que:

* El entorno TRAINING mantiene todas las variables, incluyendo Cover_Type, la cual será utilizada como variable objetivo durante el entrenamiento del modelo.

* El entorno SERVING excluye la variable Cover_Type, garantizando que el modelo, una vez desplegado en producción, no tenga acceso a esta variable y deba inferirla en función de las características de entrada.

In [27]:
# Verificar entornos definidos en el esquema
print("Entornos disponibles en el esquema:", schema.default_environment)

# Verificar qué características NO están en SERVING
for feature in schema.feature:
    if "SERVING" in feature.not_in_environment:
        print(f"'{feature.name}' no está en SERVING")

Entornos disponibles en el esquema: ['TRAINING', 'SERVING']
'Cover_Type' no está en SERVING


Una alternativa para verificar la correcta configuración de los entornos dentro del esquema es generar una muestra representativa del subconjunto de entrenamiento y analizar sus estadísticas mediante la función **generate_statistics_from_csv**.

Este enfoque permite realizar una validación empírica de las actualizaciones aplicadas en el esquema, asegurando que las modificaciones, como la exclusión de la variable Cover_Type en el entorno SERVING, se reflejen correctamente en los datos procesados.

Al utilizar generate_statistics_from_csv, se obtiene una descripción estadística de la muestra generada, lo que facilita la inspección de la estructura de los datos y su alineación con los entornos configurados dentro del SchemaGen. Este procedimiento garantiza que los datos se comporten según lo esperado en cada etapa del pipeline, asegurando la integridad y coherencia del flujo de datos desde el entrenamiento hasta el despliegue del modelo.

In [28]:
# Definimos la estructura de las variables que tenemos dentro del conjunto de datos
feature_description = {
    'Hillshade_Noon': tf.io.FixedLenFeature([], tf.int64),
    'Horizontal_Distance_To_Fire_Points': tf.io.FixedLenFeature([], tf.int64),
    'Horizontal_Distance_To_Roadways': tf.io.FixedLenFeature([], tf.int64),
    'Wilderness_Area': tf.io.FixedLenFeature([], tf.string),
    'Slope': tf.io.FixedLenFeature([], tf.int64),
    'Elevation': tf.io.FixedLenFeature([], tf.int64),
    'Horizontal_Distance_To_Hydrology': tf.io.FixedLenFeature([], tf.int64),
    'Cover_Type': tf.io.FixedLenFeature([], tf.string),
    'Soil_Type': tf.io.FixedLenFeature([], tf.string),
    'Hillshade_9am': tf.io.FixedLenFeature([], tf.int64),
}

# Definimos una función auxiliar que permita leer la muestra con la estructura correcta de la información
def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

# Tomamos la muestra del conjunto de entrenamiento 
prep_dataset = dataset.map(_parse_function)
data_list = [example for example in prep_dataset.take(100000)] 
pd_data_serving = pd.DataFrame(data_list)

# Definimos una función auxiliar para la lectura serializada de los datos
def tensor_to_value(x):
    if isinstance(x, tf.Tensor):
        return x.numpy().decode("utf-8") if x.dtype == tf.string else x.numpy()
    return x

# Construimos el dataframe dentro de Jupyterlab 
pd_data_serving = pd_data_serving.applymap(tensor_to_value).drop("Cover_Type", axis=1)

# Definir la ruta donde se guardará el archivo CSV
data_dir = os.path.join(os.getcwd(), "data_tfx")
os.makedirs(data_dir, exist_ok=True)
csv_path_serving = os.path.join(data_dir, "data_serving.csv")

# Guardar el DataFrame como CSV (sin el índice para evitar problemas)
pd_data_serving.to_csv(csv_path_serving, index=False)

2025-03-04 02:28:21.432856: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Se procede a la verificación de la muestra seleccionada del conjunto de entrenamiento con el objetivo de evaluar si la ausencia de la característica **Cover_Type** en el entorno SERVING afecta la generación de estadísticas desde el archivo CSV.

A través del análisis de las estadísticas generadas, se confirma que la exclusión de la variable objetivo no compromete la capacidad del **pipeline** para calcular métricas descriptivas sobre las demás características. Esto valida la correcta configuración del esquema y refuerza la solidez del flujo de datos en cada fase del pipeline, garantizando que el modelo podrá operar en un entorno de producción sin depender de Cover_Type.

Este procedimiento es crucial para asegurar que la transición entre entrenamiento y despliegue se realice de manera consistente, minimizando el riesgo de errores o desajustes en la interpretación de los datos procesados.

In [29]:
serving_stats = tfdv.generate_statistics_from_csv(csv_path_serving)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema, environment = 'SERVING')
tfdv.display_anomalies(serving_anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Se ejecuta la verificación del esquema utilizando la muestra generada en el entorno de entrenamiento (TRAINING), con el objetivo de evaluar la presencia de la variable **Cover_Type**. En este análisis, se observa que la ausencia de esta característica genera un conflicto dentro del esquema, lo que confirma que la variable es indispensable en el contexto de entrenamiento del modelo.

Este resultado valida nuevamente que la configuración de los entornos fue aplicada correctamente, ya que:

* En **TRAINING**, Cover_Type es esencial, dado que representa la variable objetivo del modelo. Su ausencia genera inconsistencias en la estructura esperada del conjunto de datos.

* En **SERVING**, la eliminación de Cover_Type es intencional y no afecta el cálculo de estadísticas ni la estructura de los datos, asegurando un flujo de inferencia adecuado en producción.

Esta validación reafirma la solidez del **pipeline**, garantizando que los datos sean interpretados correctamente en cada etapa del proceso, desde el entrenamiento hasta el despliegue del modelo en un entorno de producción.

In [30]:
serving_stats = tfdv.generate_statistics_from_csv(csv_path_serving)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema, environment = 'TRAINING')
tfdv.display_anomalies(serving_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


Esta verificación es un paso fundamental en la configuración del pipeline, ya que asegura la correcta transición entre las fases de entrenamiento y despliegue, preservando la coherencia en el flujo de datos y evitando filtraciones de información que podrían comprometer la validez del modelo en un entorno real.

## 3.7. Genere Nuevas Estadísticas Usando el Esquema Actualizado

Se procede a la importación del esquema previamente generado con el propósito de garantizar la consistencia en la estructura de los datos en futuras ejecuciones del **pipeline**. Posteriormente, se lleva a cabo una nueva generación de estadísticas utilizando el esquema actualizado, lo que permite evaluar la correcta integración de los ajustes realizados en las etapas previas.

Esta nueva ejecución de StatisticsGen facilita la validación de que las modificaciones en el esquema, tales como la diferenciación de entornos y la redefinición de restricciones en las variables, se reflejan correctamente en el procesamiento de los datos.

In [31]:
from tfx.v1.components import ImportSchemaGen
schema_gen = ImportSchemaGen(schema_file=schema_uri + "/schema.pbtxt")

# Ejecutar el componente para actualizarlo
context.run(schema_gen)

# Crear StatisticsGen pasando el esquema actualizado
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema']  # Pasamos el esquema actualizado
)

# Ejecutar el componente en el contexto de TFX
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Durante la regeneración de estadísticas en el **pipeline** de TFX, se observó que la variable **Cover_Type** seguía siendo interpretada como un dato de tipo **INT64**, en lugar de **STRING**, a pesar de los ajustes previos en el esquema.  

Al examinar el artefacto **SchemaGen**, se confirmó que la estructura de la variable no había cambiado, manteniendo su formato numérico. Esto se debe a que **TFX asigna automáticamente los tipos de datos según el análisis estadístico del conjunto de datos**, categorizando como **INT64** aquellas características cuyos valores son exclusivamente numéricos, incluso si conceptualmente representan etiquetas o categorías.  

Este comportamiento resalta la **limitación de los módulos actuales de TFX**, los cuales permiten inferir el esquema pero no transformar directamente los datos dentro del pipeline. Por ello, **es indispensable modificar el conjunto de datos antes de su ingreso a TFX**, asegurando que **Cover_Type** se almacene como **STRING** desde el inicio. Esto garantiza que el pipeline interprete correctamente la variable como categórica, en lugar de un valor numérico, preservando la coherencia del esquema y evitando errores en las etapas posteriores del procesamiento.

In [32]:
context.show(statistics_gen.outputs['statistics'])

## 3.8. Comprobar Anomalías

Para la detección y validación de anomalías dentro de los subconjuntos de entrenamiento y validación, se emplea el componente **ExampleValidator** de TFX. Este artefacto compara los datos de entrada con el esquema definido en SchemaGen, permitiendo identificar discrepancias estructurales, valores atípicos o inconsistencias en el conjunto de datos.

Tras la ejecución de **ExampleValidator**, se confirma que no se detectaron anomalías en ninguno de los subconjuntos procesados. Este resultado indica que las modificaciones aplicadas en el artefacto **SchemaGen** han sido correctamente integradas, garantizando que los datos cumplen con los estándares definidos en el esquema curado.

La ausencia de errores o discrepancias valida la coherencia del flujo de datos dentro del **pipeline**, asegurando que el conjunto de datos procesado está alineado con la estructura esperada, lo que permite continuar con las siguientes etapas del experimento sin riesgo de sesgos o inconsistencias en la información.

In [33]:
from tfx.components import ExampleValidator

# Crear el componente ExampleValidator con estadísticas y esquema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],  # Estadísticas generadas
    schema=schema_gen.outputs['schema']  # Esquema actualizado
)

# Ejecutar ExampleValidator en el pipeline de TFX
context.run(example_validator)

# Mostrar las anomalías detectadas
context.show(example_validator.outputs['anomalies'])

## 3.9. Ingeniería de Características

En esta sección, se procede a la creación del módulo **covertype_constants.py**, el cual servirá como un repositorio centralizado de constantes y configuraciones, facilitando la integración de las transformaciones dentro del pipeline de TFX.

La principal finalidad de este módulo es proporcionar una estructura organizada y reutilizable para las operaciones de transformación, permitiendo su uso y compatibilidad con la función **preprocessing_fn**, la cual será desarrollada en etapas posteriores. Esta función será clave para la ingeniería de características, asegurando que los datos sean normalizados, convertidos y preprocesados de manera consistente antes de ser utilizados en el entrenamiento del modelo.

El uso de un archivo de configuración independiente como **covertype_constants.py** permite mejorar la modularidad del código, facilitando modificaciones futuras sin afectar directamente la implementación del pipeline.

In [34]:
# Definición del conjunto de definición de variables y sus transformaciones
_covertype_constants_module_file = 'covertype_constants.py'

In [35]:
%%writefile {_covertype_constants_module_file}

# Características no numéricas - categoricas
CATEGORICAL_FEATURE_KEYS = ['Soil_Type' , 'Wilderness_Area']

# Características numéricas para hacer estalamiento min-max 0-1
NUMERIC_TO_SCALE = ["Hillshade_9am","Hillshade_Noon"]

# Características numéricas para hacer escalamiento por estandarización gaussiana
NUMERIC_TO_Z = ["Elevation","Horizontal_Distance_To_Fire_Points","Horizontal_Distance_To_Hydrology","Horizontal_Distance_To_Roadways"]

# Características a la cual se le pueden hacer binning (generación de buckets)
BUCKET_FEATURE_KEYS = ['Slope']

# Número de buckets a realizar 
FEATURE_BUCKET_COUNT = {'Slope': 5}

# Característica de interés, variable a predecir
Cover_Type_KEY = 'Cover_Type'

# Función auxiliar para llamar las variables transformadas
def transformed_name(key):
    return key + '_xf'

Writing covertype_constants.py


## 3.10. Función de Preprocesamiento

En esta sección, se procede a la definición de la función **preprocessing_fn**, cuya principal responsabilidad es aplicar transformaciones y preprocesamientos a las variables previamente seleccionadas dentro del **pipeline** de TFX.

Esta función es clave dentro del componente **Transform**, ya que permite realizar operaciones de ingeniería de características, garantizando que los datos sean procesados y estructurados de manera óptima antes de ser utilizados en el entrenamiento del modelo. Algunas de las principales tareas que pueden incluirse dentro de esta función son:

* **Escalado entre 0 y 1** de las variables, donde una variable numérica se estandariza en números entre 0 y 1.

* **Normalización** de variables numéricas para garantizar que todas las características tengan una escala comparable.

* **Codificación de variables categóricas**, transformándolas en representaciones numéricas adecuadas para modelos de aprendizaje automático.

* **Conversión y transformación** de datos en función de reglas definidas previamente en el módulo covertype_constants.py.

El diseño de preprocessing_fn asegura la coherencia del preprocesamiento a lo largo del pipeline, permitiendo que tanto los datos de entrenamiento como los de inferencia sean transformados de la misma manera, lo que garantiza estabilidad y reproducibilidad en el desempeño del modelo.

In [36]:
# Definición del conjunto de definición de variables y sus transformaciones
_covertype_transform_module_file = 'covertype_transform.py'

In [37]:
%%writefile {_covertype_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import covertype_constants

# Materializamos las variables definidas para las correspondientes transformaciones ya predefinidas
_CATEGORICAL_FEATURE_KEYS=covertype_constants.CATEGORICAL_FEATURE_KEYS
_NUMERIC_TO_SCALE=covertype_constants.NUMERIC_TO_SCALE
_NUMERIC_TO_Z=covertype_constants.NUMERIC_TO_Z
_BUCKET_FEATURE_KEYS=covertype_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT=covertype_constants.FEATURE_BUCKET_COUNT
_Cover_Type_KEY=covertype_constants.Cover_Type_KEY
_transformed_name=covertype_constants.transformed_name

# Definimos las transformaciones
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Escalamiento min-max hacía [0,1]
    for key in _NUMERIC_TO_SCALE:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])

    # Estandarización gaussiana, media 0 y desviación estándar 1
    for key in _NUMERIC_TO_Z:
        outputs[_transformed_name(key)] = tft.scale_to_z_score(
            inputs[key])
    
    # Binning - (generación de buckets)
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Conversión de cadenas string en índices
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # convertir la variable de interes a índices
    outputs[_transformed_name(_Cover_Type_KEY)] = tft.compute_and_apply_vocabulary(inputs[_Cover_Type_KEY])
    
    return outputs


Writing covertype_transform.py


## 3.11. Transformar

Se procede a la implementación del componente **Transform** dentro del **pipeline** de **TFX**, aplicando la función de preprocesamiento (**preprocessing_fn**) previamente definida. Este componente es fundamental en la ingeniería de características, ya que permite transformar los datos brutos en representaciones más óptimas para el entrenamiento del modelo.

La ejecución de Transform dentro del pipeline garantiza que:

-  Las transformaciones definidas en preprocessing_fn se apliquen de manera sistemática a todo el conjunto de datos.

-  Se generen versiones transformadas y enriquecidas de las características, mejorando la capacidad predictiva del modelo.

-  Se mantenga la coherencia en el preprocesamiento, asegurando que las mismas transformaciones sean utilizadas tanto en entrenamiento como en inferencia.

Este paso es clave para optimizar la calidad de los datos, reducir sesgos y garantizar que el modelo reciba características adecuadamente estructuradas para su entrenamiento y posterior despliegue en un entorno de producción.

In [38]:
# Ignorar los errores de tensorflow
tf.get_logger().setLevel('ERROR')

# Inicializar el componente Transform
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_covertype_transform_module_file)
)

# Correr el componente
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying covertype_transform.py -> build/lib
copying covertype_constants.py -> build/lib
installing to /tmp/tmp1_vv778a
running install
running install_lib
copying build/lib/covertype_transform.py -> /tmp/tmp1_vv778a
copying build/lib/covertype_constants.py -> /tmp/tmp1_vv778a
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp1_vv778a/tfx_user_code_Transform-0.0+644dd3fce6ed48486d014ca10aebef6455c95b1246f45cb5ac0b

/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+644dd3fce6ed48486d014ca10aebef6455c95b1246f45cb5ac0b5e93231d11f1-py3-none-any.whl



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+644dd3fce6ed48486d014ca10aebef6455c95b1246f45cb5ac0b5e93231d11f1-py3-none-any.whl



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+644dd3fce6ed48486d014ca10aebef6455c95b1246f45cb5ac0b5e93231d11f1-py3-none-any.whl



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Se procede a la inspección y recuperación de la estructura de almacenamiento generada por el módulo Transform, con el objetivo de analizar los artefactos creados durante la ejecución del proceso de transformación de características.

Dentro de esta estructura se almacenan elementos clave, tales como:

-  Esquema de características transformadas, el cual define la estructura de las variables después del preprocesamiento.

-  Mapeos de transformación, que incluyen las funciones aplicadas a cada variable.

-  Ejemplos transformados, en formato TFRecords, listos para ser utilizados en las siguientes etapas del pipeline.

Este análisis es fundamental para validar que las transformaciones aplicadas mediante preprocessing_fn han sido correctamente ejecutadas, asegurando la integridad y coherencia del flujo de datos dentro del pipeline de TFX.

In [39]:
# obtener el uri de la gráfica de transform 
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
# listar los subdirectorios de la uri de transform
print(os.listdir(transform_graph_uri))

['metadata', 'transform_fn', 'transformed_metadata']


In [40]:
# Obtener los uri de la salidad de la transformación de la representación de los artefactos
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Obtener la lista de los archivos almacenados como TFRecord
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

# Crear un TFRecordDataset para estos archivos
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Obtenemos 3 ejemplos de estas transformaciones
sample_records_xf = get_records(transformed_dataset, 3)

# Imprimimos los resultados de este proceso
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.11579835]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [-0.30818257]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [-0.95168304]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-0.8517184]}},
                           'Slope_xf': {'int64List': {'value': ['0']}},
                           'Soil_Type_xf': {'int64List': {'value': ['1']}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['1']}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevatio

2025-03-04 02:29:18.942307: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# 4. Metadatos de Aprendizaje Automático

En esta etapa, trabajamos con **ML Metadata** (**MLMD**), un componente esencial dentro de **TFX** que permite la exploración y análisis detallado del **pipeline**. MLMD facilita el seguimiento del **flujo de trabajo**, proporcionando acceso a los artefactos generados, ejecuciones realizadas y dependencias entre componentes.

Uno de los aspectos clave de esta integración es la capacidad de consultar, validar y analizar la evolución del pipeline, lo que resalta la importancia de haber guardado correctamente el estado del pipeline. Esto permite:

-  Rastrear la trazabilidad de los datos, identificando cómo cada artefacto ha sido producido.

-  Auditar las ejecuciones del pipeline, asegurando la coherencia en las diferentes etapas de procesamiento.

-  Depurar y optimizar el pipeline, permitiendo el análisis detallado de los componentes y su rendimiento.

A continuación, se establece la conexión con MLMD y se definen un conjunto de funciones especializadas, diseñadas para realizar consultas sobre los artefactos almacenados, inspeccionar ejecuciones previas y obtener información estructurada del pipeline. Esto nos permitirá realizar una exploración profunda del flujo de datos y facilitar el seguimiento detallado del pipeline en entornos de experimentación y producción.

In [41]:
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2
print('MLMD version: {}'.format(mlmd.__version__))

MLMD version: 1.16.0


In [42]:
connection_config = context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

In [43]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

## 4.1. Acceso a Artefactos Almacenados 

Una de las funcionalidades clave de ML Metadata (MLMD) es la capacidad de consultar y analizar los tipos de artefactos almacenados dentro del pipeline. Esto permite obtener una visión estructurada de los componentes generados a lo largo del flujo de procesamiento de datos, facilitando la auditoría, validación y optimización del pipeline.

Al explorar los tipos de artefactos disponibles, podemos identificar elementos como:

-  **ExampleGen:** Contiene los datos ingeridos y transformados en formato TFRecords.

-  **SchemaGen:** Representa el esquema inferido a partir de los datos procesados.

-  **StatisticsGen:** Almacena las estadísticas calculadas para cada subconjunto de datos.

-  **TransformGraph:** Guarda la estructura de transformación aplicada a los datos.

-  **Trainer:** Contiene los modelos entrenados junto con sus métricas de desempeño.


Esta exploración permite:

-  Verificar la completitud del pipeline, asegurando que se han generado todos los artefactos esperados.

-  Realizar un análisis detallado del flujo de datos, identificando dependencias entre los componentes.

-  Optimizar el almacenamiento y ejecución del pipeline, eliminando artefactos obsoletos o innecesarios.

A continuación, se ejecuta la consulta sobre MLMD para listar todos los tipos de artefactos generados dentro del pipeline, permitiendo una exploración detallada de su estructura y contenido.

In [44]:
display_types(store.get_artifact_types())

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,21,ExampleAnomalies
4,23,TransformGraph
5,24,TransformCache


Además de identificar los artefactos almacenados en el pipeline, ML Metadata (MLMD) nos permite analizar la trazabilidad de los esquemas (SchemaGen) generados durante el experimento. Este análisis es fundamental para comprender cómo ha evolucionado la definición de los datos a lo largo del flujo de trabajo, permitiendo realizar un seguimiento detallado de los cambios aplicados en el esquema inferido y curado.

El análisis de trazabilidad de los esquemas permite:

-  Visualizar versiones previas del esquema, identificando diferencias entre el esquema inferido automáticamente y el actualizado manualmente.

-  Evaluar el impacto de las modificaciones aplicadas al esquema sobre los datos procesados.

-  Garantizar la coherencia del pipeline, asegurando que las transformaciones y validaciones se aplican de manera consistente sobre un esquema estable.

Al consultar MLMD, podemos obtener una lista cronológica de los esquemas generados, mostrando información clave como:

-  Fecha de creación del esquema.

-  Estado actual del esquema dentro del pipeline.

-  Ubicación del artefacto en el sistema de almacenamiento.

Este nivel de trazabilidad es esencial en procesos de experimentación y producción, ya que permite validar cómo han evolucionado las definiciones estructurales de los datos y facilita la identificación de posibles inconsistencias en la canalización de procesamiento de datos.

In [45]:
# Obtener todos los artefactos del tipo "Schema"
schema_artifacts = store.get_artifacts_by_type("Schema")

# Mostrar la información de los artefactos obtenidos
#df_schema = 
display_artifacts(store, schema_artifacts)

,artifact id,type,uri
0,3,Schema,./SchemaGen/schema/3
1,4,Schema,./ImportSchemaGen/schema/4
2,10,Schema,./Transform/pre_transform_schema/7
3,12,Schema,./Transform/post_transform_schema/7


Además de analizar la trazabilidad de los artefactos generados, ML Metadata (MLMD) nos permite profundizar en el estudio de los atributos y propiedades almacenados en cada artefacto dentro del pipeline. Cada artefacto en TFX contiene un conjunto de propiedades predefinidas y propiedades personalizadas que proporcionan información clave sobre su origen, uso y estado dentro del flujo de trabajo.

El análisis de las propiedades de los artefactos permite:

-  Examinar metadatos esenciales, como el nombre, el estado del artefacto y el componente que lo generó.

-  Identificar relaciones entre artefactos, explorando qué entradas y salidas han sido utilizadas en cada componente del pipeline.

-  Realizar auditorías y depuraciones avanzadas, asegurando que cada artefacto ha sido generado con los parámetros esperados.

Dentro de las propiedades más relevantes que podemos consultar en un artefacto encontramos:

-  **name:** Identificador del artefacto dentro del pipeline.

-  **state:** Estado actual del artefacto (PENDING, COMPLETE, FAILED, etc.).

-  **producer_component:** Nombre del componente de TFX que generó el artefacto.

-  **split_names (si aplica):** Indica los subconjuntos de datos para los que se ha generado el artefacto (ejemplo: train, eval).

Explorar estas propiedades nos brinda una visión más detallada del funcionamiento del pipeline, facilitando la validación de su estructura y asegurando que cada artefacto ha sido generado y utilizado correctamente dentro del flujo de procesamiento de datos.

In [46]:
# Obtener todos los artefactos del tipo "ExampleStatistics" (puedes cambiar a otro tipo si deseas)
example_statistics_artifacts = store.get_artifacts_by_type("ExampleStatistics")

# Seleccionar el primer artefacto de la lista (puedes cambiar el índice si deseas otro artefacto)
artifact = example_statistics_artifacts[0]

# Mostrar las propiedades del artefacto seleccionado
df_properties = display_properties(store, artifact)
print(display(df_properties))

,property,value
0,span,0
1,split_names,"[""train"", ""eval""]"
2,name,statistics:2025-03-04T02:27:56.254877
3,tfx_version,1.16.0
4,sample_rate_by_split,0
5,producer_component,StatisticsGen
6,stats_dashboard_link,


None


## 4.2. Seguimiento de Artefactos

Otro aspecto fundamental del análisis con ML Metadata (MLMD) es la posibilidad de rastrear la genealogía de los artefactos, permitiéndonos identificar qué artefactos contribuyeron a la creación de otro. En este caso, realizamos el seguimiento del único TransformGraph generado en nuestro experimento, lo que nos permitirá comprender su origen y las dependencias dentro del pipeline.

El análisis de la trazabilidad de artefactos permite:

-  Determinar las entradas utilizadas en la generación del TransformGraph, tales como el esquema (SchemaGen), las estadísticas (StatisticsGen) y los datos de entrenamiento (ExampleGen).

-  Confirmar que los artefactos utilizados en la transformación corresponden a las versiones correctas, evitando inconsistencias en el pipeline.

-  Facilitar auditorías y depuraciones, asegurando que el flujo de datos ha seguido la estructura esperada y que no se han omitido pasos en el proceso de transformación.

Al ejecutar esta consulta sobre MLMD, podemos recuperar y visualizar los artefactos que fueron utilizados como insumos para la generación del TransformGraph, verificando la coherencia del pipeline y proporcionando un marco detallado para el seguimiento del flujo de datos dentro de TFX.

In [47]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

artifact_TransformGraph = store.get_artifacts_by_type("TransformGraph")#[0]

parent_artifacts = get_one_hop_parent_artifacts(store, artifact_TransformGraph)
artifact_seg = display_artifacts(store, parent_artifacts)
print(display(artifact_seg))

,artifact id,type,uri
0,1,Examples,./CsvExampleGen/examples/1
1,4,Schema,./ImportSchemaGen/schema/4


None


## 4.3. Obtener Artefactos Principales

Se ha desarrollado una función que permite trazar los artefactos predecesores de un artefacto específico dentro del **pipeline** de **TFX**, facilitando la inspección del flujo de datos y la relación entre los diferentes componentes del sistema.

En este caso, se ha aplicado la función para recuperar los artefactos que contribuyeron a la generación de un artefacto **TransformGraph** particular. Este análisis es fundamental para comprender el historial de procesamiento de datos dentro del pipeline, permitiendo:

-  Identificar las fuentes de datos que fueron utilizadas en el proceso de transformación.

-  Confirmar la coherencia del flujo de datos, asegurando que cada artefacto ha sido generado a partir de las entradas esperadas.

-  Facilitar la depuración y validación del pipeline, al proporcionar una estructura clara sobre cómo cada artefacto ha sido derivado.


La salida de este análisis proporciona información detallada sobre los artefactos predecesores, incluyendo su ID, tipo, y ubicación dentro del almacenamiento del pipeline. Esto permite verificar de manera efectiva cómo los diferentes componentes interactúan en la construcción del pipeline de TFX, asegurando un procesamiento estructurado y reproducible de los datos.

In [48]:
def get_upstream_artifacts(store, artifact):
    """
    Obtiene los artefactos que se usaron para generar un artefacto específico.
    Args:
        store: Objeto MetadataStore de MLMD.
        artifact: Artefacto de MLMD para el cual se buscan los artefactos predecesores.
    Returns:
        DataFrame con los artefactos que se usaron para generar el artefacto dado.
    """
    # Obtener todas las ejecuciones relacionadas con este artefacto
    events = store.get_events_by_artifact_ids([artifact.id])
    # Obtener los IDs de las ejecuciones que generaron este artefacto
    execution_ids = {event.execution_id for event in events if event.type == metadata_store_pb2.Event.OUTPUT}
    # Obtener todos los eventos asociados con esas ejecuciones
    upstream_events = store.get_events_by_execution_ids(list(execution_ids))
    # Filtrar los eventos que corresponden a INPUT (es decir, artefactos previos usados)
    input_artifact_ids = {event.artifact_id for event in upstream_events if event.type == metadata_store_pb2.Event.INPUT}
    # Obtener los artefactos de entrada
    upstream_artifacts = store.get_artifacts_by_id(list(input_artifact_ids))
    # Construir un DataFrame con la información de los artefactos previos
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in upstream_artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)

# Obtener todos los artefactos del tipo "TransformGraph"
transform_graph_artifacts = store.get_artifacts_by_type("TransformGraph")
transform_graph = transform_graph_artifacts[0]
df_upstream = get_upstream_artifacts(store, transform_graph)
print(display(df_upstream))


,artifact id,type,uri
0,1,Examples,./CsvExampleGen/examples/1
1,4,Schema,./ImportSchemaGen/schema/4


None
